# Abstract

This is a clone of the script at https://www.kaggle.com/ceshine/lgbm-starter which is intended to give an idea of how to structure the data for trainig

# Prelude 

## Configuration

In [1]:
DataSetPath = "/home/bryanfeeney/Dropbox/OttomanDiviner/datasets/favorita/"

StoresPath   = DataSetPath + "stores.csv.gz"
ItemsPath    = DataSetPath + "items.csv.gz"
OilPricePath = DataSetPath + "oil.csv.gz"
HolidaysPath = DataSetPath + "holidays_events.csv.gz"
Transactions = DataSetPath + "transactions.csv.gz"
TrainData    = DataSetPath + "train.csv.gz"
TestData     = DataSetPath + "test.csv.gz"

## Imports

In [2]:
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Intro to the Data

In [74]:
df_train = pd.read_csv(
    TrainData, 
    usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909),  # 2016-01-01
    compression='gzip'
)

df_test = pd.read_csv(
    TestData,
    usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"],  # , date_parser=parser
    compression='gzip'
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [31]:
items = pd.read_csv(
    ItemsPath,
).set_index("item_nbr")

stores = pd.read_csv(
    StoresPath
).set_index("store_nbr")

In [5]:
df_train.shape

(59038132, 5)

In [6]:
df_test.shape

(3370464, 2)

In [7]:
items.shape

(4100, 3)

## Select only 2017

This is a peculiar one, and it **games the benchmark** in a not great way. Essentially it uses the last 11 weeks of data before the prediction threshold to predict what's happening next

In [22]:
df_2017 = df_train[df_train.date.isin(
    pd.date_range("2017-05-31", periods=7 * 11))].copy()


In [11]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
50912462,2017-05-31,1,96995,0.693147,False
50912463,2017-05-31,1,99197,0.693147,False
50912464,2017-05-31,1,103520,1.386294,False
50912465,2017-05-31,1,103665,2.197225,False
50912466,2017-05-31,1,105574,1.386294,False


In [14]:
df_train.iloc[0,0], df_train.iloc[-1,0]

(Timestamp('2016-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))

In [15]:
df_2017.iloc[0,0], df_2017.iloc[-1,0]

(Timestamp('2017-05-31 00:00:00'), Timestamp('2017-08-15 00:00:00'))

In [27]:
df_2017.shape

(8125670, 5)

In [21]:
YearsInTrain = (2017-2013)+1
    
(df_train.shape[0]/YearsInTrain)*(11/52)

2497767.123076923

In [23]:
# del df_train

## Creating Promotion Variables

So this is a tricky. If one presumes that on-promotion will lead to a boost in demand, if if we presume we'll know *whats on promotion in advance*, then we can create variables to say that this product will be on promotion 1, 2, 3, ... 16 days from now (16 days in the future is the target)

In this case, this is also peculiar, there is a column for every single day!

In [38]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

In [39]:
promo_2017_train.head()

onpromotion
store_nbr item_nbr date                   
1         96995    2017-05-31        False
          99197    2017-05-31        False
          103520   2017-05-31        False
          103665   2017-05-31        False
          105574   2017-05-31        False

In [40]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)



In [41]:
promo_2017_train.head()

onpromotion                                              \
date                2017-05-31 2017-06-01 2017-06-02 2017-06-03 2017-06-04   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False      False      False      False   

                                                                           \
date               2017-06-05 2017-06-06 2017-06-07 2017-06-08 2017-06-09   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                      ...                                                  \
date                  ...     2017-08-06 2017-08-07 2017-08-08 2017-08-09   
store_nbr item_nbr    ...                                                   
1         96995       ...          False      False      False      False   
          99197       ...          False      False      False      False   
          103520      ...          False      False      False      False   
          103665      ...          False      False      False      False   
          105574      ...          False      False      False      False   

                                                                           \
date               2017-08-10 2017-08-11 2017-08-12 2017-08-13 2017-08-14   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                               
date               2017-08-15  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False  

[5 rows x 77 columns]

In [42]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)


In [43]:
promo_2017.shape, items.shape[0] * stores.shape[0]

((156790, 93), 221400)

In [44]:
promo_2017.head()

date                2017-05-31  2017-06-01  2017-06-02  2017-06-03  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-06-04  2017-06-05  2017-06-06  2017-06-07  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-06-08  2017-06-09     ...      2017-08-22  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-23  2017-08-24  2017-08-25  2017-08-26  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-27  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 93 columns]

In [45]:
del promo_2017_test, promo_2017_train

#  Unstack unit sales - do it across all days in a sliding window

Ah... they're creating a multi-task learning problem

In [46]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)
df_2017.shape

(156790, 77)

In [47]:
df_2017.head()

date                2017-05-31  2017-06-01  2017-06-02  2017-06-03  \
store_nbr item_nbr                                                   
1         96995       0.693147    0.000000    0.000000    0.000000   
          99197       0.693147    1.386294    1.098612    1.945910   
          103520      1.386294    1.098612    1.098612    0.693147   
          103665      2.197225    0.000000    1.791759    1.791759   
          105574      1.386294    2.484907    1.791759    1.386294   

date                2017-06-04  2017-06-05  2017-06-06  2017-06-07  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       1.098612    1.098612    0.000000    0.000000   
          103520      0.000000    0.693147    1.609438    0.693147   
          103665      1.098612    1.386294    1.791759    1.386294   
          105574      1.386294    1.386294    2.079442    2.397895   

date                2017-06-08  2017-06-09     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.693147     ...        1.098612   
          99197       0.693147    0.693147     ...        0.000000   
          103520      0.693147    1.098612     ...        0.000000   
          103665      0.000000    1.098612     ...        0.693147   
          105574      1.945910    2.079442     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 77 columns]

## Make items match other data frames

They're sacraficing generability

In [48]:
items = items.reindex(df_2017.index.get_level_values(1))
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1
105574,GROCERY I,1045,0


In [49]:
items.shape

(156790, 3)

## Time futzing

In [50]:
# Return that portion of the data frame that corresponds to the time period
#   beginning "minus" days before "dt" and extending for "periods" days
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [51]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({  # Mean target for different retrospective timespans & total # promotions
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values
    })
    for i in range(16):  # Promotions on future days
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[  # Target values for future days
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [52]:
print("Preparing dataset...")
t2017 = date(2017, 6, 21)
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [53]:
X_train

,mean_14_2017,mean_3_2017,mean_7_2017,promo_14_2017,promo_0,promo_1,promo_2,promo_3,promo_4,promo_5,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,0.099021,0.000000,0.099021,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.835944,1.245890,0.987960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.840554,0.462098,0.773092,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.141420,1.059351,1.243926,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.645124,1.416165,1.505723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.250081,2.507286,2.380866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0.688537,0.462098,0.526983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0.297063,0.000000,0.297063,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1.070138,0.231049,0.973349,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1.818947,2.132310,2.019905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
X_train.shape, y_train.shape

((627160, 20), (627160, 16))

This dataset is **super gamey**. They're using the means for the week, fortnight, and last three days, and then seeing how to permute it to generate values for the following window of time. It's hardcoded to product IDs, not categories.

It does however, permit multi-task learning, and therefore better representation learning

It does not incorporate any information about seasonality at all, and so would fall arse over face at Christmas



In [57]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

Training and predicting models...


In [58]:
MAX_ROUNDS = 1000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 4) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

Step 1


/home/bryanfeeney/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.345147	valid_1's l2: 0.341487
[100]	training's l2: 0.333379	valid_1's l2: 0.330174
[150]	training's l2: 0.331405	valid_1's l2: 0.328823
[200]	training's l2: 0.330344	valid_1's l2: 0.328317
[250]	training's l2: 0.329476	valid_1's l2: 0.327889
[300]	training's l2: 0.328793	valid_1's l2: 0.327649
[350]	training's l2: 0.328187	valid_1's l2: 0.327459
[400]	training's l2: 0.327652	valid_1's l2: 0.327329
[450]	training's l2: 0.327151	valid_1's l2: 0.327218
[500]	training's l2: 0.326681	valid_1's l2: 0.327129
[550]	training's l2: 0.326264	valid_1's l2: 0.327102
[600]	training's l2: 0.325878	valid_1's l2: 0.327031
[650]	training's l2: 0.325453	valid_1's l2: 0.326991
[700]	training's l2: 0.325065	valid_1's l2: 0.326944
[750]	training's l2: 0.324698	valid_1's l2: 0.326981
Early stopping, best iteration is:
[712]	training's l2: 0.324969	valid_1's l2: 0.326938
mean_14_2017: 1984359.03
mean_7_2017: 1460047.19
mean_3_

Step 7
Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.399249	valid_1's l2: 0.499368
[100]	training's l2: 0.387491	valid_1's l2: 0.484385
[150]	training's l2: 0.384487	valid_1's l2: 0.483271
[200]	training's l2: 0.382909	valid_1's l2: 0.482962
[250]	training's l2: 0.381732	valid_1's l2: 0.482566
[300]	training's l2: 0.380894	valid_1's l2: 0.482571
Early stopping, best iteration is:
[265]	training's l2: 0.381465	valid_1's l2: 0.482494
mean_14_2017: 2176120.92
mean_7_2017: 788873.90
mean_3_2017: 309822.71
promo_6: 154213.49
promo_14_2017: 50427.60
promo_3: 14230.01
promo_7: 10208.19
promo_13: 8896.97
promo_5: 7975.26
promo_0: 4785.81
promo_1: 4559.44
promo_4: 4185.97
promo_9: 3828.04
promo_2: 3637.22
promo_14: 3187.51
promo_8: 2329.80
promo_11: 1460.21
promo_15: 1382.96
promo_12: 1252.57
promo_10: 1153.89
Step 8
Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.387984	valid_1's l2: 0.460446
[100]	training's l2: 0.37561

Step 15
Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.410998	valid_1's l2: 0.410436
[100]	training's l2: 0.397157	valid_1's l2: 0.398619
[150]	training's l2: 0.394315	valid_1's l2: 0.397761
[200]	training's l2: 0.392755	valid_1's l2: 0.397444
[250]	training's l2: 0.391625	valid_1's l2: 0.397205
[300]	training's l2: 0.390647	valid_1's l2: 0.397032
[350]	training's l2: 0.389848	valid_1's l2: 0.396838
[400]	training's l2: 0.38916	valid_1's l2: 0.396693
[450]	training's l2: 0.388521	valid_1's l2: 0.396705
Early stopping, best iteration is:
[418]	training's l2: 0.388918	valid_1's l2: 0.396637
mean_14_2017: 2326294.16
mean_7_2017: 697294.28
mean_3_2017: 242807.08
promo_14: 232132.33
promo_14_2017: 49620.78
promo_7: 31713.90
promo_0: 29744.91
promo_15: 19946.17
promo_13: 11068.67
promo_9: 9069.83
promo_12: 8493.43
promo_10: 6377.01
promo_2: 4226.62
promo_6: 4122.19
promo_8: 3039.53
promo_11: 3002.61
promo_4: 2528.08
promo_1: 1724.67
promo_3: 1156.68
promo

In [65]:
print("Validation mse:", mean_squared_error(
    np.expm1(y_val), np.expm1(np.array(val_pred)).transpose()))

Validation mse: 275.7862813287469


In [63]:
val_pred

[array([0.2071499 , 0.48488404, 0.73822611, ..., 0.23754703, 2.05418453,
        0.03076805]),
 array([0.20248389, 0.50000804, 0.68828877, ..., 0.2134666 , 1.93274017,
        0.04175205]),
 array([0.23902936, 0.62783665, 0.75723977, ..., 0.23085787, 1.77148465,
        0.0464756 ]),
 array([0.30655785, 0.75207269, 0.99879814, ..., 0.32088842, 2.08985238,
        0.07291537]),
 array([0.27424474, 0.69718693, 1.04320573, ..., 0.36169774, 1.96112359,
        0.07797592]),
 array([0.22258805, 0.59696122, 0.81102486, ..., 0.26593051, 1.76900857,
        0.04240829]),
 array([0.21105402, 0.57495346, 0.74622924, ..., 0.24370768, 0.07047951,
        0.09334097]),
 array([0.19939098, 0.58468629, 0.73820218, ..., 0.22899961, 1.87168311,
        0.09038593]),
 array([0.21100979, 0.57101541, 0.68382262, ..., 0.21276259, 1.69936576,
        0.08189474]),
 array([0.2239681 , 0.66356936, 0.73385747, ..., 0.23369309, 1.77138193,
        0.06100372]),
 array([0.31060667, 0.78366861, 0.98043279, ..., 0

In [60]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)

Making submission...


In [61]:
submission

id  unit_sales
store_nbr item_nbr date                             
1         96995    2017-08-16  125497040    0.273266
          99197    2017-08-16  125497041    0.225388
          103501   2017-08-16  125497042    0.000000
          103520   2017-08-16  125497043    0.648250
          103665   2017-08-16  125497044    1.352113
          105574   2017-08-16  125497045    3.222305
          105575   2017-08-16  125497046    7.469437
          105576   2017-08-16  125497047    0.000000
          105577   2017-08-16  125497048    0.294655
          105693   2017-08-16  125497049    0.319934
          105737   2017-08-16  125497050    0.763024
          105857   2017-08-16  125497051    3.247849
          106716   2017-08-16  125497052    1.781587
          108079   2017-08-16  125497053    0.346823
          108634   2017-08-16  125497054    0.033705
          108696   2017-08-16  125497055    1.357327
          108698   2017-08-16  125497056    0.585320
          108701   2017-08-16  125497057    1.804330
          108786   2017-08-16  125497058    1.744146
          108797   2017-08-16  125497059    2.840066
          108831   2017-08-16  125497060    0.000000
          108833   2017-08-16  125497061    0.000000
          108862   2017-08-16  125497062    0.704088
          108952   2017-08-16  125497063    1.000221
          111223   2017-08-16  125497064    3.262282
          111397   2017-08-16  125497065    0.431402
          112830   2017-08-16  125497066    1.090800
          114778   2017-08-16  125497067    1.617434
          114790   2017-08-16  125497068    2.928812
          114799   2017-08-16  125497069    0.731032
...                                  ...         ...
54        2127921  2017-08-31  128867474    0.000000
          2127992  2017-08-31  128867475    0.000000
          2128628  2017-08-31  128867476    0.000000
          2128799  2017-08-31  128867477    0.000000
          2129334  2017-08-31  128867478    0.000000
          2129350  2017-08-31  128867479    0.000000
          2129387  2017-08-31  128867480    0.000000
          2129515  2017-08-31  128867481    0.000000
          2129616  2017-08-31  128867482    0.000000
          2129678  2017-08-31  128867483    0.000000
          2129786  2017-08-31  128867484    0.000000
          2129790  2017-08-31  128867485    0.000000
          2129892  2017-08-31  128867486    0.000000
          2129994  2017-08-31  128867487    0.000000
          2130131  2017-08-31  128867488    0.000000
          2130219  2017-08-31  128867489    0.000000
          2130265  2017-08-31  128867490    0.000000
          2130352  2017-08-31  128867491    0.000000
          2130474  2017-08-31  128867492    0.000000
          2130521  2017-08-31  128867493    0.000000
          2130526  2017-08-31  128867494    0.000000
          2130553  2017-08-31  128867495    0.000000
          2131010  2017-08-31  128867496    0.000000
          2131572  2017-08-31  128867497    0.000000
          2131699  2017-08-31  128867498    0.000000
          2132163  2017-08-31  128867499    0.000000
          2132318  2017-08-31  128867500    0.000000
          2132945  2017-08-31  128867501    0.000000
          2132957  2017-08-31  128867502    0.000000
          2134244  2017-08-31  128867503    0.000000

[3370464 rows x 2 columns]

# Further Improvements

This is based on the work in this file: https://www.kaggle.com/vrtjso/lgbm-one-step-ahead

This was apparently in the top 10% at one point.

In [76]:
df_train = pd.read_csv(
    TrainData, usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    TestData, usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    ItemsPath,
).set_index("item_nbr")

df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

print("Training and predicting models...")
params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)


Preparing dataset...
Training and predicting models...
Step 1


/home/bryanfeeney/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.30191	valid_1's l2: 0.29409
[200]	training's l2: 0.298363	valid_1's l2: 0.292741
[300]	training's l2: 0.295918	valid_1's l2: 0.292337
[400]	training's l2: 0.293791	valid_1's l2: 0.2921
[500]	training's l2: 0.29199	valid_1's l2: 0.29195
Did not meet early stopping. Best iteration is:
[500]	training's l2: 0.29199	valid_1's l2: 0.29195
mean_7_2017: 1882639.38
mean_14_2017: 1229821.77
promo_0: 104143.51
day_1_2017: 89857.46
mean_20_dow0_2017: 84245.48
mean_3_2017: 76646.29
mean_30_2017: 76583.57
mean_4_dow0_2017: 58919.38
mean_60_2017: 33035.18
promo_14_2017: 28619.72
promo_7: 9432.45
mean_4_dow5_2017: 7417.05
mean_140_2017: 7406.32
promo_60_2017: 6740.72
mean_20_dow4_2017: 5611.55
promo_140_2017: 5493.72
mean_4_dow6_2017: 4633.44
mean_4_dow2_2017: 3813.74
mean_20_dow2_2017: 3343.78
mean_4_dow3_2017: 2824.66
promo_9: 2814.25
mean_4_dow1_2017: 2707.00
mean_20_dow3_2017: 2642.58
mean_20_dow1_2017: 2616.99
me

Step 7
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.346219	valid_1's l2: 0.421254
[200]	training's l2: 0.341874	valid_1's l2: 0.420923
[300]	training's l2: 0.339035	valid_1's l2: 0.420617
[400]	training's l2: 0.336652	valid_1's l2: 0.420437
Early stopping, best iteration is:
[393]	training's l2: 0.336799	valid_1's l2: 0.420378
mean_14_2017: 1274004.42
mean_30_2017: 842088.63
mean_7_2017: 445757.89
mean_20_dow6_2017: 152802.16
mean_3_2017: 145241.24
promo_6: 128192.88
mean_4_dow6_2017: 127503.94
mean_60_2017: 123326.90
promo_14_2017: 21944.55
day_1_2017: 13872.06
promo_3: 11205.46
promo_7: 9013.03
mean_4_dow5_2017: 8533.91
mean_20_dow5_2017: 8280.78
promo_60_2017: 8112.57
promo_140_2017: 6018.04
mean_20_dow1_2017: 4884.13
promo_5: 4487.41
mean_140_2017: 4382.31
promo_13: 3877.01
mean_4_dow0_2017: 3650.61
mean_20_dow0_2017: 3639.82
mean_20_dow3_2017: 3507.37
mean_4_dow1_2017: 3355.21
mean_20_dow4_2017: 2943.12
promo_4: 2784.87
promo_0: 2663.62
mean

[200]	training's l2: 0.365615	valid_1's l2: 0.376333
[300]	training's l2: 0.362327	valid_1's l2: 0.376258
Early stopping, best iteration is:
[338]	training's l2: 0.361236	valid_1's l2: 0.37609
mean_30_2017: 1601076.04
mean_14_2017: 588977.50
mean_60_2017: 374936.35
mean_7_2017: 305452.67
mean_3_2017: 148957.18
promo_12: 93518.03
mean_20_dow5_2017: 85159.77
mean_4_dow5_2017: 71638.34
promo_13: 19595.91
promo_14_2017: 16011.22
promo_14: 13355.62
promo_10: 11265.36
mean_140_2017: 9927.43
promo_60_2017: 8798.33
day_1_2017: 7950.62
mean_20_dow0_2017: 7544.02
promo_140_2017: 6088.78
mean_20_dow6_2017: 5888.75
mean_4_dow6_2017: 4725.31
promo_11: 4479.14
mean_4_dow0_2017: 3756.38
mean_20_dow3_2017: 3664.43
promo_9: 3613.91
mean_20_dow2_2017: 3341.99
mean_4_dow2_2017: 2714.48
mean_20_dow1_2017: 2701.22
mean_4_dow3_2017: 2566.89
mean_20_dow4_2017: 2531.39
promo_15: 2495.30
mean_4_dow1_2017: 2301.79
mean_4_dow4_2017: 2274.90
promo_7: 2189.77
promo_0: 1734.63
promo_8: 1699.81
promo_5: 1614.77
prom

In [77]:
print("Validation mse:", mean_squared_error(
    np.expm1(y_val), np.expm1(np.array(val_pred)).transpose()))

Validation mse: 247.36471602948214


In [78]:
np.sqrt(275), np.sqrt(247)

(16.583123951777, 15.716233645501712)